In [1]:
import os
import cudf
import math
import numpy as np
import pandas as pd
import cupy as cp
from math import cos, sin, asin, sqrt, pi, atan2
from numba import cuda

In [2]:
np.random.seed(12)

In [3]:
!nvidia-smi

Tue Apr  5 14:35:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P3    15W /  N/A |     58MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'data-old.csv',
 'data.csv',
 'Untitled.ipynb',
 'us-accidents-rapids-notebook.ipynb']

In [5]:
print ("Size of file is {} MB".format(os.path.getsize("data.csv") >> 20))

Size of file is 1101 MB


In [6]:
%time df = cudf.read_csv('data.csv')
print(df)

CPU times: user 3.98 s, sys: 7.77 s, total: 11.8 s
Wall time: 57.8 s
                ID  Severity           Start_Time             End_Time  \
0              A-1         3  2016-02-08 00:37:08  2016-02-08 06:37:08   
1              A-2         2  2016-02-08 05:56:20  2016-02-08 11:56:20   
2              A-3         2  2016-02-08 06:15:39  2016-02-08 12:15:39   
3              A-4         2  2016-02-08 06:51:45  2016-02-08 12:51:45   
4              A-5         3  2016-02-08 07:53:43  2016-02-08 13:53:43   
...            ...       ...                  ...                  ...   
2845337  A-2845338         2  2019-08-23 18:03:25  2019-08-23 18:32:01   
2845338  A-2845339         2  2019-08-23 19:11:30  2019-08-23 19:38:23   
2845339  A-2845340         2  2019-08-23 19:00:21  2019-08-23 19:28:49   
2845340  A-2845341         2  2019-08-23 19:00:21  2019-08-23 19:29:42   
2845341  A-2845342         2  2019-08-23 18:52:06  2019-08-23 19:21:31   

         Start_Lat   Start_Lng    End_Lat 

In [7]:
df.describe()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,2.845342e+06,2.845342e+06,2.845342e+06,2.845342e+06,2.845342e+06,2.845342e+06,1.101431e+06,2.776068e+06,2.375699e+06,2.772250e+06,2.786142e+06,2.774796e+06,2.687398e+06,2.295884e+06
mean,2.137572e+00,3.624520e+01,-9.711463e+01,3.624532e+01,-9.711439e+01,7.026780e-01,8.089408e+03,6.179356e+01,5.965823e+01,6.436545e+01,2.947234e+01,9.099391e+00,7.395044e+00,7.017000e-03
std,4.787220e-01,5.363797e+00,1.831782e+01,5.363873e+00,1.831763e+01,1.560361e+00,1.836009e+04,1.862263e+01,2.116097e+01,2.287457e+01,1.045286e+00,2.717546e+00,5.527454e+00,9.348800e-02
min,1.000000e+00,2.456603e+01,-1.245481e+02,2.456601e+01,-1.245457e+02,0.000000e+00,0.000000e+00,-8.900000e+01,-8.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.344517e+01,-1.180331e+02,3.344628e+01,-1.180333e+02,5.200000e-02,1.270000e+03,5.000000e+01,4.600000e+01,4.800000e+01,2.931000e+01,1.000000e+01,3.500000e+00,0.000000e+00
50%,2.000000e+00,3.609861e+01,-9.241808e+01,3.609799e+01,-9.241772e+01,2.440000e-01,4.007000e+03,6.400000e+01,6.300000e+01,6.700000e+01,2.982000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.016024e+01,-8.037243e+01,4.016105e+01,-8.037338e+01,7.640000e-01,9.567000e+03,7.600000e+01,7.600000e+01,8.300000e+01,3.001000e+01,1.000000e+01,1.000000e+01,0.000000e+00
max,4.000000e+00,4.900058e+01,-6.711317e+01,4.907500e+01,-6.710924e+01,1.551860e+02,9.999997e+06,1.960000e+02,1.960000e+02,1.000000e+02,5.890000e+01,1.400000e+02,1.087000e+03,2.400000e+01


In [8]:
len(df)

2845342

In [9]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 2845342 entries, 0 to 2845341
Data columns (total 47 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Severity               int64
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Description            object
 10  Number                 float64
 11  Street                 object
 12  Side                   object
 13  City                   object
 14  County                 object
 15  State                  object
 16  Zipcode                object
 17  Country                object
 18  Timezone               object
 19  Airport_Code           object
 20  Weather_Timestamp      object
 21  Temperature(F)         float64
 22  Wind_Chill(F)          float64
 23  

In [10]:
df.isna().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1743911
Street                         2
Side                           0
City                         137
County                         0
State                          0
Zipcode                     1319
Country                        0
Timezone                    3659
Airport_Code                9549
Weather_Timestamp          50736
Temperature(F)             69274
Wind_Chill(F)             469643
Humidity(%)                73092
Pressure(in)               59200
Visibility(mi)             70546
Wind_Direction             73775
Wind_Speed(mph)           157944
Precipitation(in)         549458
Weather_Condition          70636
Amenity   

In [11]:
df = df.drop(columns = ['ID','Start_Time','End_Time','Street','Side','Description',
                        'Number','City','Country','Zipcode','Timezone',
                        'Airport_Code','Weather_Timestamp','Wind_Chill(F)',
                        'Wind_Direction','Wind_Speed(mph)','Precipitation(in)'])

In [12]:
df.isna().sum()

Severity                     0
Start_Lat                    0
Start_Lng                    0
End_Lat                      0
End_Lng                      0
Distance(mi)                 0
County                       0
State                        0
Temperature(F)           69274
Humidity(%)              73092
Pressure(in)             59200
Visibility(mi)           70546
Weather_Condition        70636
Amenity                      0
Bump                         0
Crossing                     0
Give_Way                     0
Junction                     0
No_Exit                      0
Railway                      0
Roundabout                   0
Station                      0
Stop                         0
Traffic_Calming              0
Traffic_Signal               0
Turning_Loop                 0
Sunrise_Sunset            2867
Civil_Twilight            2867
Nautical_Twilight         2867
Astronomical_Twilight     2867
dtype: int64

In [13]:
df['Temperature(F)'] = df['Temperature(F)'].fillna(df['Temperature(F)'].mean())
df['Humidity(%)'] = df['Humidity(%)'].fillna(df['Humidity(%)'].mean())
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())

df['Weather_Condition'] = df['Weather_Condition'].fillna('Fair')
df['Sunrise_Sunset'] = df['Sunrise_Sunset'].fillna('Day')
df['Civil_Twilight'] = df['Civil_Twilight'].fillna('Day')
df['Nautical_Twilight'] = df['Nautical_Twilight'].fillna('Day')
df['Astronomical_Twilight'] = df['Astronomical_Twilight'].fillna('Day')

In [14]:
df.drop(df.tail(1).index,inplace=True) 
df.isna().sum()

Severity                 0
Start_Lat                0
Start_Lng                0
End_Lat                  0
End_Lng                  0
Distance(mi)             0
County                   0
State                    0
Temperature(F)           0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal           0
Turning_Loop             0
Sunrise_Sunset           0
Civil_Twilight           0
Nautical_Twilight        0
Astronomical_Twilight    0
dtype: int64

In [15]:
def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 

        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_2 = pi/180 * x_2
        y_2 = pi/180 * y_2
        
        dlon = y_2 - y_1
        dlat = x_2 - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_2) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        out[i] = c * r

In [16]:
%%time

df = df.apply_rows(haversine_distance_kernel,
                   incols=['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng'],
                   outcols=dict(out=np.float64),
                   kwargs=dict())

CPU times: user 954 ms, sys: 0 ns, total: 954 ms
Wall time: 1.05 s


In [17]:
def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 

        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_2 = pi/180 * x_2
        y_2 = pi/180 * y_2
        
        dlon = y_2 - y_1
        dlat = x_2 - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_2) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        out[i] = c * r

In [18]:
%%time

outdf = df.apply_chunks(haversine_distance_kernel,
                        incols=['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng'],
                        outcols=dict(out=np.float64),
                        kwargs=dict(),
                        chunks=8,
                        tpb=8)

CPU times: user 809 ms, sys: 0 ns, total: 809 ms
Wall time: 1.49 s


In [19]:
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),County,State,Temperature(F),Humidity(%),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,out
0,3,40.108910,-83.092860,40.112060,-83.031870,3.230,Franklin,OH,42.1,58.0,...,False,False,False,False,False,Night,Night,Night,Night,5.198541
1,2,39.865420,-84.062800,39.865010,-84.048730,0.747,Montgomery,OH,36.9,91.0,...,False,False,False,False,False,Night,Night,Night,Night,1.201714
2,2,39.102660,-84.524680,39.102090,-84.523960,0.055,Hamilton,OH,36.0,97.0,...,False,False,False,False,False,Night,Night,Night,Day,0.088753
3,2,41.062130,-81.537840,41.062170,-81.535470,0.123,Summit,OH,39.0,55.0,...,False,False,False,False,False,Night,Night,Day,Day,0.198752
4,3,39.172393,-84.492792,39.170476,-84.501798,0.500,Hamilton,OH,37.0,93.0,...,False,False,False,False,False,Day,Day,Day,Day,0.805093


In [20]:
outdf.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),County,State,Temperature(F),Humidity(%),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,out
0,3,40.108910,-83.092860,40.112060,-83.031870,3.230,Franklin,OH,42.1,58.0,...,False,False,False,False,False,Night,Night,Night,Night,5.198541
1,2,39.865420,-84.062800,39.865010,-84.048730,0.747,Montgomery,OH,36.9,91.0,...,False,False,False,False,False,Night,Night,Night,Night,1.201714
2,2,39.102660,-84.524680,39.102090,-84.523960,0.055,Hamilton,OH,36.0,97.0,...,False,False,False,False,False,Night,Night,Night,Day,0.088753
3,2,41.062130,-81.537840,41.062170,-81.535470,0.123,Summit,OH,39.0,55.0,...,False,False,False,False,False,Night,Night,Day,Day,0.198752
4,3,39.172393,-84.492792,39.170476,-84.501798,0.500,Hamilton,OH,37.0,93.0,...,False,False,False,False,False,Day,Day,Day,Day,0.805093


In [21]:
df.to_csv("processed.csv", index=False)